In [26]:
#Lession 1 - 서울시 응답소 사이트에 접속 후 메뉴 클릭
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import math
import pandas as pd  
import openpyxl
import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 크롤링에 필요한 정보를 입력 받습니다.
print("=" *100)
print("연습문제: 서울시 응답소 게시판 크롤링하기")
print("=" *100)

query_txt = '서울시응답소'
cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):")
if f_dir=='' :
    f_dir ='c:\\py_temp\\'
    
print()

# 저장될 위치와 이름을 지정합니다
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)
ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

#Step 3. 크롬드라이버 설정 -> 응답소 사이트 접속 -> 메뉴클릭
s_time = time.time( )  # 시작시간 스탑워치 클릭

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get('https://eungdapso.seoul.go.kr/')
driver.maximize_window()

#페이지가 다 열릴 때까지 2-5초 사이 기다리기
time.sleep(random.randrange(2,5))  

#시장에게 바랍니다 메뉴 클릭하기
driver.find_element(By.XPATH,'//*[@id="gnb"]/div[4]/div[2]/div/a').click()

#Lesson 2 - 게시물 상세 정보 수집하기
# Step 4.게시물의 정보 수집

no2=[]             # 게시글 번호 컬럼
title2=[]          # 게시글 제목 컬럼
odate2=[]          # 게시글 공개일 컬럼
contents2=[]       # 게시글 상담내역 컬럼

no = 1    # 게시글 번호 카운트 변수

for x in range(1,page_cnt+1) :
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
        
    # 각 페이지에 있는 게시글 10건의 상세 내역 가져오기
    for i in range(2,12) :
        
        if no > cnt :
            break
        else :
           
            #txt 형식으로 저장할 파일 지정
            f = open(ff_name, 'a',encoding='UTF-8')           

            # 게시글 제목 클릭
            print('===== %s번째 게시물의 상세내역을 추출합니다=====' %no)
            driver.find_element(By.XPATH,'//*[@id="content_cont"]/div[2]/div/form/div[4]/table/tbody/tr[%s]/td[2]/a' %i).click( )
            time.sleep(1)
                               
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
           
            # 1.번호 입력
            no2.append(no)
            
            # 2.제목 추출
            title_1 = soup.find('table').find('tbody').find_all('tr')
            title = title_1[0].get_text().replace('제목','').replace('\n','')
            print('1.제목:' , title)
            f.write('1.제목:' + title + '\n' )
            title2.append(title)
            
            # 3.공개일 추출
            odate = title_1[1].get_text().replace('공개일','').replace('\n','')
            print('2.공개일:', odate)
            f.write('2.공개일:' + odate + '\n' )
            odate2.append(odate)
                       
            # 4.상담내역 추출
            contents = soup.find('div','table_inner_desc').get_text().replace('Q. 상담내용','').replace('\n','').strip()
            print('3.상담내역:', contents , '\n')
            f.write('3.상담내역:' + contents +'\n')
            contents2.append(contents)
            f.close( )
            

            no += 1

            #이전 페이지로 돌아가기
            driver.back()
        
    time.sleep(1)      
    
    x += 1    
    try :
        driver.find_element(By.LINK_TEXT,'%s' %x).click() # 다음 페이지번호 클릭
    except :
        driver.find_element(By.LINK_TEXT,'>').click()

#Lesson 3 - 수집된 정보를 다양한 형식의 파일로 저장하기
# Step 6. 수집된 정보를 다양한 형식의 파일로 저장하기 
seoul = pd.DataFrame()
seoul['번호'] = no2
seoul['제목'] = title2
seoul['공개일'] = odate2
seoul['상담내용']=contents2

# csv 형태로 저장하기
seoul.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
seoul.to_excel(fx_name,index=False, engine='openpyxl')

#Step 7. 요약 정보 출력하기
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *100)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )

연습문제: 서울시 응답소 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 58
크롤링 할 총 페이지 번호:  6
2.결과 파일을 저장할 폴더명만 쓰세요(예:c:\py_temp\):

1 페이지 내용 수집 시작합니다 =======================
===== 1번째 게시물의 상세내역을 추출합니다=====
1.제목: 강남구에 버림받은 땅 ?? 일원 1동 현대화 시설 요청
2.공개일: 2022-03-10
3.상담내역: 수고하십니다.강남구에 버림받은 땅이 있습니다. 바로 일원1동입니다.일원1동 수서1단지 옆 자원회수시설에는 쓰레기처리장, 하수종말처리장, 열병합발전소 등 3대 기피시설이 집중되어 있습니다. 특히 쓰레기처리장은 서울시 8개구로 부터 수집된 쓰레기를 처리 중인데, 일원1동 주민들은 90년대 중반 이후 기피시설 수용에 따른 혜택도 전혀 체감하지 못하고 있습니다. 동일한 시설을 운영 중인 마포구 당인리 발전소나 하남 유니온시티 한번 가보시기 바랍니다. 두 곳은 테마공원으로 지역 명소화가 된 반면, 강남자원회수시설은 그에 비하면 낡은 90년대 공장 이미지입니다. 똑같은 세금내는 강남구민입니다. 왜 일원1동만 피해를 입어야하는지 모르겠습니다.당인리 발전소, 유니온시티 지하화 및 상부 공원화한 것과 똑같이 강남자원회수시설 현대화를 조속히 추진해주시기 바랍니다.시설 현대화가 어렵다면 즉각적인 시설 폐쇄 요청드립니다. 

===== 2번째 게시물의 상세내역을 추출합니다=====
1.제목: [전기차] 미성년 장애인 가구 추가 보조금 적용 件
2.공개일: 2022-03-04
3.상담내역: 시장님 안녕하세요.미성년 장애아동의 부모로서 이번 서울시 대기환경과의 '전기차 추가 보조금' 정책에 대한 아쉬움을 전해드립니다.전기차 보조금의 경우 취약계층을 대상으로 100만원의 추가 보조금이 지급되나, 미성년 장애인 가구에 대한 지원은 '우선순위' 부여 외에는 없습니다.미성년 장애인은 성인 장애인보다 부모의 도움을 받아 자차로의 이동이 더욱 필요한 약자로서 성인 장애인

2 페이지 내용 수집 시작합니다 =======================
===== 11번째 게시물의 상세내역을 추출합니다=====
1.제목: 보육 발전을 위해 애써 주심에 감사합니다.
2.공개일: 2022-02-11
3.상담내역: 서울시 보육 발전을 위해 늘 애써 주심에 감사를 드립니다.어제 서울시 보육 포털 서비스 공지 사항을 통해 서울형 전담 보조교사 신설 내용을 알게 되어 관계 부처로 문의 전화를 하며 얼마나 설레였는지 모릅니다.현장에는 아이들을 위한 많은 손이 필요합니다.더욱이 예전에 비해 발달 지연 아동의 수도 늘어가고 저희 어린이집 처럼 외국인 아동의 수가 많은 경우는 더더욱 그렇습니다.그러던 중 서울형 전담 보조교사의 소식은 단비와 같았습니다. 감사합니다.저는 서울형 어린이집을 운영하고 있지만 많은 어린이집이 정부의 지원을 받지 못하고 있는 곳들이 많아 운영의 어려움을 많이 호소하고 있습니다. 그러던 중 보육료의 인상과(아직도 표준 보육비용에는 어려움이 있다고 생각합니다- 인건비 .물가 상승을 비교하면)  반 운영비등 어린이집을 위해 서울시에서 애써 주시고 관심 가져 주신 것을 느낄 수 있어 감사한 마음 이 자리를 빌어 감사를 드립니다.앞으로도 보육에 더 큰 관심을 가져 주시고 현장의 소리를 들어 주시면 감사하겠습니다.한 가지 큰 바람이 있다면 외국인 아동 수가 많은 어린이집으로서 유치원에서 주는 교육비 지원으로 인해 아이들이 이동하여 어린이집 운영이 더 힘들어지지 않기를 간절히 또 간절히 바라는 마음입니다.늘 대한민국 보육의 선두주자인 서울시를 응원합니다. 

===== 12번째 게시물의 상세내역을 추출합니다=====
1.제목: 장애인 지하철 엘리베이터 설치
2.공개일: 2022-02-09
3.상담내역: 모든 정차역에 장애인 엘리베이터 설치해주세요일반인이 편하게 걸어 다닐수 있는 거처럼 장애인분들도 눈치 안보고 더이상 사고 나지 않고 다닐 수 있는 장애인 엘리베이터 꼭 설치 해 주세요코로나라고 모든 자영업자 엄청 퍼주는 거의 반의 반만이라도 

1.제목: 광화문을 금융허브로
2.공개일: 2022-01-12
3.상담내역: 한양도성안의 1)삼성그룹 현대그룹 등은 강남으로, 2)행정은 세종시로 떠나버렸습니다. 3)관광관련산업도 대부분 문을 닫았습니다. 길을 잃었습니다.도심 공동화 현상이 심각하고, 관련 자영업 소상공인의 폐업이 줄을 잇고 있으며, 일자리가 급감하고 있습니다.다행히 지역 전통 금융산업인 한국은행, 미래에셋, 하나외환은행 등과 다국적기업 서울사무소가 명맥을 유지하고 있습니다. 한편 홍콩은 해가 저물고 있어 금융기업 다국적기업 등이 다른 지역으로 이전하거나 준비하고 있습니다. 일부는 광화문을 선택합니다.런던 뉴욕 홍콩 싱가포르와 같이 광화문을 세계적인 금융허브와 다국적기업 비즈니스센터로 집중 육성할 절호의 기회입니다.광화문을 홍콩과 같이 세계적인 금융비즈니스센터로 육성하기 위해 시장님께서 직접 추진위원장을 맡아주시면 참 좋겠습니다.감사합니다. 

===== 20번째 게시물의 상세내역을 추출합니다=====
1.제목: C19 극복을 위한 [안심업장] 인증 구조
2.공개일: 2022-01-11
3.상담내역: C19 극복을 위한 [안심업장] 인증 구조식당의 예를 들면 테이블별로 구역을 묶어 투명 아크릴로 분리하여 감염 발생이 있는 경우 그 구역만 폐쇄하고 감염과 무관한 구역은 계속 영업하기 위한 고안임1. 공간 ABC는 각각 밀폐된 독립공간이며 독립된 출입문이 있다.2. 각 객실에는 건물 외부에 연결된 맑은 공기를 유입한다.3. 각 실내 공기는 팬으로 강제 배출된다. 4. 현재 운영 중인 불고깃집 연기 배출 구조에 아크릴로 칸막이만 하고 외부 공기 유입 설비만 각 실에 추가했다고 생각하면 된다.5. C19 감염이 발생한 경우 어느 객실인지 확인하여 그 객실만 폐쇄하고 영업은 계속하면 된다.   - 기대효과 -1. 식당 외에 노래방 피시방 등 여러 업장에 확대 적용할 수 있다.2. 우선 서울시 또는 수도권만이라도 조례를 만들어 실행된다면 절박한 소상공인들 삶에 큰 도움이 될 것이다. 

3 페이지 내용 

4 페이지 내용 수집 시작합니다 =======================
===== 31번째 게시물의 상세내역을 추출합니다=====
1.제목: 강북구 수유12구역 공공주도3080을 반대합니다
2.공개일: 2021-12-21
3.상담내역: 서울시장님저는 강북구 수유12구역에 살고 있는 주민입니다.본인은 이 지역에 평생살고자  하는 주민입니다.본인의 집에는 4가구의 독거노인들이 거주하고 있습니다,수유12구역은 갈곳이 없는  어려운 이웃들이 살기 좋은 곳입니다.몸이 편찮으신 분들.거동이 불편하신분들이 맘편히 살고 계십니다.느닷없는 공공주도3080의 발표로 이렇게 어려운 독거노인들은 어디로 쫓겨납니까더이상 갈곳이 없습니다.본인집에 거주하는 독거노인들은 공공주도3080으로 잠을 못이루고 매일 걱정과 불안에 병이 났습니다.이렇게 어려운 이웃들을 마음 편하게 살수 있도록 해주십시요,공공주도3080을 막아주십시요.수유12구역을 이대로 살수 있도록 해주십시요. 

===== 32번째 게시물의 상세내역을 추출합니다=====
1.제목: 의료비 지원 확대
2.공개일: 2021-12-16
3.상담내역: 장애인 복지카드를 가지고 있어도 수급권자가 아니면 의료비 지출이 많이 부담스럽습니다. 장애인 의료비 지원을 늘려주시기 바랍니다 

===== 33번째 게시물의 상세내역을 추출합니다=====
1.제목: 항체치료제 투약 센터를 서울시 광장에 만들어 주세요.
2.공개일: 2021-12-03
3.상담내역: 서울시 광장과 대학병원 주차장에 콘테이너 박스 설치해서 항체치료제 투약센타를 만들어 주십시오. 렉키로나 처방 권고에 해당하는 환자부터 무조건 렉키로나 투여하고 집에서 재택하도록 만드세요.병상부족, 확진자 증가세, 사망자 수 등을 크게 개선시킬 수 있습니다.그리고 요즘 국민들이 코로나 검사 받으려고 추운데 줄서서 기다리고 그러던데...... 검사하러 오신 분들 집에서 검사할 수 있게 자가진단키트 무료로 제공해주세요자가진단키드 받은 사람을 핸드폰 어풀 깔아서 등록하고 검사 결과를 핸드폰으로 

1.제목: 반지하 노후빌라 재개발 되도록 고도제한 풀어주세요
2.공개일: 2021-11-08
3.상담내역: 안녕 하십니까?저는 강북구 소유주 입니다. 저희 집은 반지하이고 너무 오래되어 수도관이 자주 터져서 집안 곳곳에 곰팡이천국 입니다.반지하방이라 햇빛도 없고 신선한 공기도 없습니다. 강북구 빨래골에 와 보시면 아시겠지만 가파른 골목골목 소방차들어오지도 못합니다.박원순시장시절 도시재생사업한다고 해서 변한건 하나도 없습니다.전 이번 오세훈시장님을 믿습니다.고도제한도 풀어주어 재개발이 될 수 있도록 도와 주세요저희 집으로 직접오셔서 반지하 집상태을 보시면 좋겠습니다. 

5 페이지 내용 수집 시작합니다 =======================
===== 41번째 게시물의 상세내역을 추출합니다=====
1.제목: 주제: 1종 일반주거지역
2.공개일: 2021-11-08
3.상담내역: 제목: 등촌2동 정비사업 대책 마련을 촉구합니다.등촌2동은 이번에 도심 공공주택 복합사업을 신청했습니다. 그런데 서울시 도시계획과에서 1종 일반주거지역이 많아 안 된다고 합니다. 그런데, 공공주택 업무지침에 따르면 ‘인근 지역과 도시계획적 연속성’에 따라서 예외적으로 1종 일반주거지역을 포함할 수 있습니다. 수색14구역 같은 경우 85%가 1종입니다. 등촌2동은 옆에 아이파크가 20층이고, 등촌1구역 재건축은 15층으로 힐스테이트가 올라갑니다. 왜 평지인 우리 동네만 1종으로 저층주거지로 남아있어야 합니까? (구 국군수도통합병원 이전 함)대로변이고, 강북횡단선도 생기는데, 이런 곳에 주택공급을 해야하는 것 아닙니까?주택공급을 위한 적극적인 행정을 요청드립니다. 

===== 42번째 게시물의 상세내역을 추출합니다=====
1.제목: 강서구 등촌2동 수도물이  녹물이 나옴
2.공개일: 2021-11-08
3.상담내역: 30년이  넘는  빌 라  날마다 녹물을  마시며  생활을  하고 있습니다.이러다  암이라도  걸리면  서울시에서 우리가족 책임을 지실  건지  대답이  듣고  싶습니다

1.제목: 오세훈 시장님의 공약인 신안산선 대림삼거리역 출입구추가
2.공개일: 2021-10-18
3.상담내역: 오세훈 시장님께서 공약이셨던 대림삼거리역 출입구 추가 신설을 해주신다고 하셨는데어떻게 진행이 되고 있는지 궁굼합니다.국토교통부에 문의를 하니 서울시에서 별다른 언급이 없었다고 하던데요.대림삼거리역 인근 아파트 단지 주민들은 공약을 보고 투표하신분들이 대부분입니다.(신길경남아너스빌, 신대방우성1차, 신대방 현대힐스테이트, e편한세상 영등포 아델포레, 신길우성2차, 3차, 우창아파트, 신길삼성아파트 등)그동안 신안산선 대림삼거리역 위치를 원안대로 설치해달라고 힘모아 요청했습니다만비용의 문제로 현재 대림사거리쪽으로 많이 치우쳐있는 상태입니다.결국 구로디지털단지역 <700m> 대림삼거리역 <1.3km> 신풍역이런 기이한 역간 거리가 생겨 버린 이유가 결국 돈때문이었다고 해명합니다.오세훈 시장님께서 추가로 출입구를 신설 해주기로 하셨으니 부디 답변 부탁드립니다. 

===== 57번째 게시물의 상세내역을 추출합니다=====
1.제목: 발상의 전환 필요. 소각장 건설이 능사가 아님. 인구
2.공개일: 2021-10-18
3.상담내역: 발상의 전환이 필요합니다.소각장을 고집하지 마십시오.소각장 건설이 능사가 아닙니다.소각장을 건설하고 운영하면 대기환경오염, 주민건강 위협, 그로 인한 재정적 부담이 높아집니다.소각장에서 나오는 온실가스 양이 석탄발전소보다 더 많습니다.소각장에서 나오는 다이옥신, NOx, Sulfur, 미세먼지, 수은, 납 등의 유해물질이 대기를 오염시키고 온난화를 가속화시키며 주변 주민들의 건강을 해칩니다.소각할 때 나오는 낙진, 재와 폐수(침출수 포함)sms 환경과 주민에게 막대한 악영향을 미칩니다.왜 소각장만을 찾으십니까? 다른 대안을 모색하십시오!인구는 계속 줄고 있습니다. 많지 않은 인구들은 계속 도시로 몰리고 있고 지방은 공동화되고 있습니다.소각하지 말고 차라리 다른 매립지를 찾으십시오!다른 매립지를 찾아 적절한 보상을 하고 매립하기 전